In [5]:
# Replace with your ADLS Gen2 account name, container name, and file path
adls_account_name = "ecommerceanalytics23"
adls_container_name = "ecommerce-logs"
file_path = "enrichedData"

# Construct the full ADLS path
adls_path = f"abfss://{adls_container_name}@{adls_account_name}.dfs.core.windows.net/{file_path}"

try:
    # Read the CSV file into a Spark DataFrame
    df = spark.read.parquet(adls_path)

    # Display the first few rows of the DataFrame
    df.show()

except Exception as e:
    print(f"Error reading file: {e}")
    print("Please double-check your ADLS account name, container name, and file path.")

StatementMeta(ECommSparkPool, 7, 6, Finished, Available, Finished)

+-------------------+----------+----------+--------------------+--------------------+------------+-------+---------+--------------------+----+----------+
|         event_time|event_type|product_id|         category_id|       category_code|       brand|  price|  user_id|        user_session|Hour|weekNumber|
+-------------------+----------+----------+--------------------+--------------------+------------+-------+---------+--------------------+----+----------+
|2019-11-17 09:58:09|      view|  11200328|2.053013562946749...|appliances.person...|       vitek|  17.94|519234057|e5110e4c-0672-460...|   9|        46|
|2019-11-17 09:58:09|      view|   6201100|2.053013552293216...|appliances.enviro...|      carver|  64.35|520936692|02d54d04-81e2-4b9...|   9|        46|
|2019-11-17 09:58:09|      view|   1004767|2.053013555631882...|electronics.smart...|     samsung| 264.79|516001435|1a125c36-397b-419...|   9|        46|
|2019-11-17 09:58:09|      view|   3700298|2.053013565983425...|appliances.e

In [6]:
df.printSchema()

StatementMeta(ECommSparkPool, 7, 7, Finished, Available, Finished)

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: double (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- weekNumber: integer (nullable = true)



In [7]:
df.count()

StatementMeta(ECommSparkPool, 7, 8, Finished, Available, Finished)

42089570

In [8]:
df_no_nulls = df.na.drop()
df_no_nulls.count() # Display the DataFrame after dropping nulls

StatementMeta(ECommSparkPool, 7, 9, Finished, Available, Finished)

42089564

In [9]:
from pyspark.sql.functions import col, hour, weekofyear

df_no_nulls = df_no_nulls.withColumn("hour",hour(col("event_time")))  \
                            .withColumn("weekNumber",weekofyear(col("event_time")))


StatementMeta(ECommSparkPool, 7, 10, Finished, Available, Finished)

### Unique visitors per day

In [10]:
from pyspark.sql.functions import to_date, col, countDistinct 

uniqueVisitorPerDay = df_no_nulls.withColumn("event_date",to_date(col("event_time"))) \
                                    .groupBy("event_date") \
                                    .agg(countDistinct("user_id").alias("unique_visitor")) \
                                    .orderBy("event_date")

uniqueVisitorPerDay.show()

StatementMeta(ECommSparkPool, 7, 11, Finished, Available, Finished)

+----------+--------------+
|event_date|unique_visitor|
+----------+--------------+
|2019-11-01|        160828|
|2019-11-02|        167895|
|2019-11-03|        174567|
|2019-11-04|        197915|
|2019-11-05|        183132|
|2019-11-06|        179224|
|2019-11-07|        194694|
|2019-11-08|        205990|
|2019-11-09|        193528|
|2019-11-10|        200168|
|2019-11-11|        210147|
|2019-11-12|        205921|
|2019-11-13|        207896|
|2019-11-14|        236287|
|2019-11-15|        315108|
|2019-11-16|        393822|
|2019-11-17|        388098|
|2019-11-18|        226772|
|2019-11-19|        196928|
|2019-11-20|        194896|
+----------+--------------+
only showing top 20 rows



In [11]:
df_no_nulls.createOrReplaceTempView("CleanData")
spark.sql("""
    SELECT
        CAST(event_time AS DATE) AS event_date,
        COUNT(DISTINCT user_id) AS unique_visitors
    FROM
        CleanData
    GROUP BY
        CAST(event_time AS DATE)
    ORDER BY
        event_date
""").show()

StatementMeta(ECommSparkPool, 7, 12, Finished, Available, Finished)

+----------+---------------+
|event_date|unique_visitors|
+----------+---------------+
|2019-11-01|         160828|
|2019-11-02|         167895|
|2019-11-03|         174567|
|2019-11-04|         197915|
|2019-11-05|         183132|
|2019-11-06|         179224|
|2019-11-07|         194694|
|2019-11-08|         205990|
|2019-11-09|         193528|
|2019-11-10|         200168|
|2019-11-11|         210147|
|2019-11-12|         205921|
|2019-11-13|         207896|
|2019-11-14|         236287|
|2019-11-15|         315108|
|2019-11-16|         393822|
|2019-11-17|         388098|
|2019-11-18|         226772|
|2019-11-19|         196928|
|2019-11-20|         194896|
+----------+---------------+
only showing top 20 rows



### During a certain time, the users add products to their carts but don’t buy them

In [12]:
spark.sql("""
    SELECT
        DISTINCT(event_type)
    FROM
        CleanData
""").show()

StatementMeta(ECommSparkPool, 7, 13, Finished, Available, Finished)

+----------+
|event_type|
+----------+
|  purchase|
|      view|
|      cart|
+----------+



In [13]:
df_cart_event = df_no_nulls.filter(col("event_type") == "cart")
df_purchase_event = df_no_nulls.filter(col("event_type") == "purchase")

abandoned_carts = df_cart_event.alias("cart").join(df_purchase_event.alias("purchase"),
                    (col("cart.user_id") == col("purchase.user_id")) &
                    (col("cart.product_id") == col("purchase.product_id")) &
                    (col("cart.user_session") == col("purchase.user_session")),"left_anti")  \
                    .select(
                        col("cart.user_id").alias("user_id"),
                        col("cart.product_id").alias("product_id"),
                        col("cart.category_id").alias("category_id"),
                        col("cart.brand").alias("brand"),
                        col("cart.event_time").alias("event_time")
                    ).distinct()

abandoned_carts.show()

StatementMeta(ECommSparkPool, 7, 14, Finished, Available, Finished)

+---------+----------+--------------------+----------+-------------------+
|  user_id|product_id|         category_id|     brand|         event_time|
+---------+----------+--------------------+----------+-------------------+
|272811580|   3601425|2.053013563810775...|      beko|2019-11-08 05:51:33|
|361978955|   1005212|2.053013555631882...|   samsung|2019-11-28 06:48:43|
|428799958|   1005135|2.053013555631882...|     apple|2019-11-25 04:59:51|
|487489100|   1005169|2.053013555631882...|   samsung|2019-11-26 12:52:48|
|497612854|  10200838|2.053013553224352E18|   llorens|2019-11-13 09:02:19|
|501799046|   6701163|2.053013554247762...|       msi|2019-11-11 14:52:18|
|512374782|   4802273|2.053013554658803...|   samsung|2019-11-17 07:40:24|
|512395943|   6000227|2.053013560807654...|  starline|2019-11-16 15:16:41|
|512396363|  28401155|2.053013566209917...|     karya|2019-11-15 02:08:39|
|512413617|   5100854|2.053013553341792...|     apple|2019-11-16 15:37:54|
|512430622|   1004839|2.0

### Top categories per hour or weekday (i.e. to promote discounts based on trends)

In [14]:
from pyspark.sql.functions import col, hour, count, desc

top_categories_hourly = df_purchase_event.groupBy("hour", "category_id") \
                                    .agg(count("*").alias("purchase_count")) \
                                    .orderBy("hour", desc("purchase_count"))
top_categories_hourly.show()

StatementMeta(ECommSparkPool, 7, 15, Finished, Available, Finished)

+----+--------------------+--------------+
|hour|         category_id|purchase_count|
+----+--------------------+--------------+
|   0|2.053013555631882...|          1200|
|   0|2.053013554658803...|           167|
|   0|2.053013553341792...|            71|
|   0|2.053013554415534...|            70|
|   0|2.053013558920217...|            62|
|   0|2.053013563810775...|            34|
|   0|2.053013565983425...|            26|
|   0|2.053013561579406...|            24|
|   0|2.172371436436455...|            20|
|   0|2.053013566176363...|            14|
|   0|2.053013555573162...|            14|
|   0|2.053013552293216...|            14|
|   0|2.053013565228450...|            13|
|   0|2.053013565069067...|            11|
|   0|2.053013565639492...|            11|
|   0|2.053013553970938...|            10|
|   0|2.053013563911439...|            10|
|   0|2.053013553945772...|            10|
|   0|2.053013560899928...|             9|
|   0|2.053013555262784E18|             8|
+----+-----

In [15]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

window_spec = Window.partitionBy("hour").orderBy(desc("purchase_count"))

top_n_categories_hourly = top_categories_hourly.withColumn("rank", rank().over(window_spec)) \
    .filter(col("rank") <= 5)  # Change 5 to your desired number of top categories
top_n_categories_hourly.show()

StatementMeta(ECommSparkPool, 7, 16, Finished, Available, Finished)

+----+--------------------+--------------+----+
|hour|         category_id|purchase_count|rank|
+----+--------------------+--------------+----+
|   0|2.053013555631882...|          1200|   1|
|   0|2.053013554658803...|           167|   2|
|   0|2.053013553341792...|            71|   3|
|   0|2.053013554415534...|            70|   4|
|   0|2.053013558920217...|            62|   5|
|   1|2.053013555631882...|          1742|   1|
|   1|2.053013554658803...|           199|   2|
|   1|2.053013554415534...|           118|   3|
|   1|2.053013563810775...|            71|   4|
|   1|2.053013553341792...|            67|   5|
|   2|2.053013555631882...|          4399|   1|
|   2|2.053013554658803...|           462|   2|
|   2|2.053013554415534...|           273|   3|
|   2|2.053013563810775...|           194|   4|
|   2|2.053013558920217...|           185|   5|
|   3|2.053013555631882...|         11015|   1|
|   3|2.053013554658803...|           966|   2|
|   3|2.053013554415534...|           70

In [21]:
from pyspark.sql.functions import col, dayofweek, count, desc, when

df_withWeekDays = df_purchase_event.withColumn("weekDay", when(dayofweek(col("event_time")) == 1, "Sunday")
                                                        .when(dayofweek(col("event_time")) == 2, "Monday")
                                                        .when(dayofweek(col("event_time")) == 3,"Tuesday")
                                                        .when(dayofweek(col("event_time")) == 4,"Wednesday")
                                                        .when(dayofweek(col("event_time")) == 5,"Thursday")
                                                        .when(dayofweek(col("event_time")) == 6, "Friday")
                                                        .otherwise("Saturday"))

top_categories_weekDay = df_withWeekDays.groupBy("weekDay", "category_id") \
                                    .agg(count("*").alias("purchase_count")) \
                                    .orderBy("weekDay", desc("purchase_count"))

window_spec_weekDay = Window.partitionBy("weekDay").orderBy(desc("purchase_count"))

top_n_categories_weekDay = top_categories_weekDay.withColumn("rank",rank().over(window_spec_weekDay))  \
                                                .filter(col("rank") <= 5)
top_categories_weekDay.show()
top_n_categories_weekDay.show()

StatementMeta(ECommSparkPool, 7, 22, Finished, Available, Finished)

+-------+--------------------+--------------+
|weekDay|         category_id|purchase_count|
+-------+--------------------+--------------+
| Friday|2.053013555631882...|         47156|
| Friday|2.053013554658803...|          4668|
| Friday|2.053013554415534...|          3162|
| Friday|2.053013563810775...|          2017|
| Friday|2.053013558920217...|          1979|
| Friday|2.053013553341792...|          1837|
| Friday|2.053013565983425...|          1648|
| Friday|2.053013563911439...|           733|
| Friday|2.172371436436455...|           673|
| Friday|2.053013561579406...|           481|
| Friday|2.053013555573162...|           476|
| Friday|2.053013554776244...|           476|
| Friday|2.053013566176363...|           444|
| Friday|2.053013555262784E18|           423|
| Friday|2.053013552293216...|           417|
| Friday|2.053013552326770...|           381|
| Friday|2.053013564003713...|           381|
| Friday|2.053013554834964...|           356|
| Friday|2.053013560807654...|    

### To know which brands need more marketing

In [29]:
from pyspark.sql.functions import col, count,avg

df_view_event = df_no_nulls.filter(col("event_type") == "view")

view_counts = df_view_event.groupBy("brand").agg(count("*").alias("view_count"))
purchase_counts = df_purchase_event.groupBy("brand").agg(count("*").alias("purchase_count"))

brand_performance = view_counts.join(purchase_counts, on="brand", how="left") \
    .na.fill(0, subset=["purchase_count"]) \
    .withColumn("conversion_rate", col("purchase_count") / col("view_count"))

average_conversion_rate = brand_performance.agg(avg("conversion_rate").alias("avg_conversion")).collect()[0]["avg_conversion"]

brands_need_marketing_conversion = brand_performance.filter(col("conversion_rate") < average_conversion_rate)

brands_need_marketing_conversion.orderBy(col("conversion_rate")).show()

StatementMeta(ECommSparkPool, 7, 30, Finished, Available, Finished)

+----------+----------+--------------+---------------+
|     brand|view_count|purchase_count|conversion_rate|
+----------+----------+--------------+---------------+
|   bompani|       205|             0|            0.0|
|dickietoys|        45|             0|            0.0|
|      tmnt|       100|             0|            0.0|
|potatohead|        14|             0|            0.0|
|     sigma|       395|             0|            0.0|
|    circle|       151|             0|            0.0|
|    mackie|        53|             0|            0.0|
|  skechers|        25|             0|            0.0|
|  funville|         2|             0|            0.0|
|  hercules|        49|             0|            0.0|
|      cata|       607|             0|            0.0|
|    kacper|       188|             0|            0.0|
|    norfin|       111|             0|            0.0|
|    makers|       681|             0|            0.0|
|    rapala|       277|             0|            0.0|
|    senco